# 01 Installation

In [ ]:
# @title Run to Install softwars
%%capture

!apt update
!apt install sra-toolkit samtools fastp fastqc unzip ncbi-blast+ htop
!pip install biopython
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.1.1/hisat2-2.2.1-64-ngs.3.1.1-linux.zip
!unzip hisat2-2.2.1-64-ngs.3.1.1-linux.zip
!rm hisat2-2.2.1-64-ngs.3.1.1-linux.zip
!mv hisat2-2.2.1-64-ngs.3.1.1-linux hisat
!wget https://github.com/ablab/spades/releases/download/v4.0.0/SPAdes-4.0.0-Linux.tar.gz
!tar -xzf SPAdes-4.0.0-Linux.tar.gz
!rm SPAdes-4.0.0-Linux.tar.gz
!mv SPAdes-4.0.0-Linux SPAdes

In [ ]:
# @title Run to Download SRA file
SRA_Run	 = 'SRR926285' # @param {type:"string"}
!prefetch $SRA_Run -p -o $SRA_Run
#!fasterq-dump $SRA_Run -p -e 8 -m 1000MB
#!fastqc *.fastq -t 8

# Mapping



In [ ]:
import os
Genome_Link	 = 'https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-59/fasta/olea_europaea/dna/Olea_europaea.OLEA9.dna.toplevel.fa.gz' # @param {type:"string"}
Genome_File = os.path.basename(Genome_Link)
!wget $Genome_Link
!mkdir Index
!mkdir Mapped
!gunzip $Genome_File
!hisat2/hisat2-build -p 8 *.fa Index/Genome
!hisat2/hisat2 -x Index/Genome -p 8 --fast --sra-acc $SRA_Run -S Mapped/Reads.sam


# None Host Reads Extracting

In [ ]:
# @title Run
!samtools view -@ 8 -bS Mapped/Reads.sam > Mapped/Reads.bam
!samtools view -@ 8 -b -f 4  Mapped/Reads.bam >  Mapped/UnM_Reads.bam
!samtools sort -n -m 5G -@ 8 Mapped/UnM_Reads.bam -o Mapped/UnM_Reads_S.bam
!samtools fastq -@ 8 Mapped/UnM_Reads_S.bam > Mapped/R1.fastq

# Assemble



In [ ]:
# @title Run
!./SPAdes/bin/spades.py --rnaviral -t 8 -s Mapped/R1.fastq -o Spades_Result/

# Detect Your Virus

In [ ]:
fasta = 'AGACCACAGCAGGAACAACTGTGTGATTCCAAAGAGAAATATCTTCAAGATAATGGCTTTCTCATACCGGACACCCGCAGAAGAGTTCCTCTCCACTTTGCCGTCAACCTCACAGGAAACAATTAGTGAATATGCACTTAAGTCACTCAAGGACCAGGAAAAGGCTTCAAGCCATAACTATTCCTATCACTTGACTGACACGCAAAAAAGCTTTTGTTCCAAGGTTGGTGTTCCCCTCTCTGTACATAGCTTCATGGTTCATCCGCACCCTTACTGTAAGACCATGGAGAATTTTATTCTGCATGACAATCTTTTTCATTATAGGAATTTGATAGAAATTTATGTTTCGATAAAGGAGGAAAAGGTTAGTTTGTTAAATAGGAGTAGGGGTTTGAATTTTCACAGGGTTTTGAATAGATGTGTTGCTGATAGGGACAATTTAAGATATAGGAATTGTGTAAGCAGATCCTTCTTTGTCAACAACAACAATTTCGCTCCCAACCAATCCAAATCATGGTTCTTTCATGATGAGCTTCACCACTGGGATGTGAACGAATTCGCTAGCTTTTTAGGGAACTTCAAACCAAAAAGGGTCATAGCTTCATGCGTGCTTCCAGTGGAGGTTTTCAGTTCTGATGAGTCTTGCAACCCCAGCTTTTACAAATACTCAATCCACAAGAGGGATAGATCACAGGTGGAATTCACTTTTTTCCCTGATAGCAGCAAAGAAAGTTCCTATTTCCAGAAAAGTTCTGACTGGATATTCAGATTCAAGCACTTTGAAATAGGAAGTGAGAACTACACCCTTTCTTTTCTGAGATCAATCAAGTGCCACCATTTGATTCTAATTGAAAAAGGGAAACTGGAAACCAATGATTTTTACATTTCAGATGAGGCTGAATGCGCATCTTTGCGAATGTTTGGGACCGCGGTGGCTGGGAAAATTGCTGCCCCAATAAGGGTGGAAGTGCTAAAAAGGGAGATGATATACCTTCTGAGCCTCAAGAAAAGCGACATGAATTCTGCCGCCGCCAAGTTAAGACAGCTCTCCCAAGAGGACTATCATCCGCAGGAGCTGTGTTTCTTTCTGAACCTTGCCGGGAGGATCGAGAGTTGCAAGGGGATCTTTAAAGACAGGGGACTAGTCTCGACCATCGTTGATTGTTTCGCTCAGATGTTTCCAAACAGAATTGCAAGGTGGATGTCCCCAACTACTGATGAGAATTCTTTCCTGCAACTGATATCCAGCATGGAGGGCATGAAACTAACAATCAAAAGAGAGTCATATCCAGTCGAGAGCTTCTTTGAATTTGATGTCAAGGATGGCCTTGATGAACAGTTTGTGCTTGACTTGATGGATGGTAAAATGTTTGGGTCTGTTTCGGAAAAACTGGATACCTCTTACTCGGTCAAAACTTCCGGTAACTGGAACTTTGTCCTCCTTGATTTCAAACAGAGGAAATGGGCCAGGTTGGCAGAGAAAGAGGAAAAGGAACCATCGGCACCAAGAGAGATCCCTGGCGAGACACAGGAGCTGTCACCTGAGGAGGCTCGATTTTATTACGGGATTCCAGGAATGAAAAGGGCCGTTCACTGCAATCTGGAATGGAGCAGGGGGAAAAGGGCATTTCAAAAGAAAAAAGACCCAATCTTCATAACCAATTTCGCACACTGGAGTCTTCCAAGAAAGGATGGAATTAGAGAAGTGGAGATGTGGGATGCTGAGGAAGAACGAACCACAGATCAATTCGTGGAAGGGTTTAATCCTGAGACGTGTATGGGTTCCAGTTCAAGTTCGGAGGCAGAGACCGAAAAGCTTGGCTCAGGGGGGAGTGGAGATGATGCACAGATGACCTCAGGGGAGGAGGAGGAGGAATGCAATTACATTAATGGCAAATTTTCTCCGAATGACACATTCATTGAGTCATTGGAAATGCTTCTGAGCTCCAGGGCCGCATATCTCCATGGTGGTAGGCGAACAATTCTTTTCTCCACAGTTACTGGCCTTGATTACGGGTTCGGGCCCCACAGATACAAGCAAGTCCCTTGCAATTTCATGGAAGATTTGCTCAAAAAGGTAGGTTTTAATAGTTGCTTGGCCCAGATCTACAAGGCCGGGGGTTCTGTTGGCGCTCACTTTGATGATGAAAAGGTGTATGATGATGATGAAATCCTCACGTGGAATCTGAAGGGTGAAGCAGACTTTTCTCTGTTTAAGAAAGGGGGAACCGAATCTGTACATCTAGGACCAAATCAGATTTTGATCATGCCAAAGGGATGTTCAAGAGGTGAGAACAGATTCAAGCACTCAGTTGAGAACTGCACAGAAGGACGTATCTCCATAACCTTTAGACACCAAAAGAGGTTCATGAATGGGGAACCCGTCGAAGTGGAGAACTTCAAAAGACAGATGATTGAACTGCCAGACAATCTATGCGATCTAATTTCAAAGATGTCGAATGCATGTTTCTTTGACTGTCTAGCTGACCACCTGTGCATGGATAGAAATGCTGTTTTCAATCTCCTTTATGACCAGGATAGATCTGTTCTGACTAACGTAATTGAGGACAAAGGCTTCACTTTGGCAGAAGTGATAGATCACCTCACGAATCTAGATGTTCCTGGCAGAATAGTGAGCAATGGTGAGGTTGTAAATTACCTGGAAAAGGGGTCATTCAAACCAATAGATTTGCTCATCAGGGATGGGCACATCGGTTTGAACGTGCAGCACAATGTGCTCTACAACACGAAGGAGATCAGGATTGACGAGTTGATCGGAGCCGACATTGTTGCACCTCACTTCTCGGTGGAGAGAGCAAGGGTATTGGTGAAGTCAATGATGGAAGGGATGACCGGAGTTCTGCTGAACAGATTTAAATATGCTTTCAATGAGCTACTCCCAAGGAATGGAAATAAGGTGATGTGCATATCGGGATTTGCTGGTTCTGGAAAATCAAGGGCTCTGCAGGGTGTATGTGCCAACGTTCTGAACAAGAAAAGCGTGATCCTGAGTAGTCCAAGGAAAAACCTTCTCAGGGACTGGGAATCAAAGATAGATGAGAAGCTAAAGGGAAAGAAACGCTTGATCAAACTCAAGACCTTTGAACTGGCCATTAGCGCCATCACGAGAATGGTCAAAGCTGAGCAGGATGCCAAATTGACGGTCATCATCGATGAAGTAACGCTCCTTCCGGGAGGCTACCTGGATCTAGTGAATAGTCTTATCCCAGAAGGGTCAACCATCATCCTACTCTTTGATCCATTGCAATCGCATTATTACTCAAGGTCTGATGTGAGCGTCAATTTGGGACCTGTTCTCACACCCTTGTTCGGTCAAGAATTTAGGTATAGAGGTTTTTCATACAGATTCCCAAAGTTGTTTGATTTGGAAGAGTTTGAGTTTGGGCAGGGTGAGGTCGATCAGAATCACATGAGGATCTTTGCTCAACCTCAAGCTGTGAGAGAGGCTATCAAGAAGCCTGTGTTTCTCTGTCCCAGTGAGGAAAAAAGATGCGAACTTTCTAACTTTGGAGAGGCATACACGTTCGGGACAAGTCAAGGTCTCACTTTTGACTTTGTTTGCATATCAATTGATATGGATGGTTCTGTGACATCGGATTTTCACTGGATGGTGGCCCTCACTCGAGCAAGGAAAGGTTTCTGCTTCCTCACATGCGCATCAGTTAGTATGAGAACTTTCATGGACAGTAACAAGGCAAAGTTGATAGGGAAAGTTCTAAAGAAAGAACAAATTTCCAAGAAGTTTTGGTGGAACTTGGGAGGCAGAGCTTTGGAGGGAGCAAGAACTGTGAAAAAAGAGGAATTCTCAAAGCTTGGGAAAACCAGGGAAGAGTTTGAGGAGGCACTGGAGGGCGACCCCTGGCTCAAGGGCATGCTGAATTACCTTGAAGGAGATGACGCAAGAGACCCGTTGCCTGACGAGCCGATTCGGAAAGAGAGTCCTCCCAGAACTCATCTGATGATTGCTCCAGTTGAACATCAATTCGCCGAAGAGATGCACCTGTTAAAAGCCAGAGAATTCAGGGAGTTCAGGAATTCCAATTTGTGGTCTGAGCAATTTGATGACGCGAGGAAAACAAGAAAAGTCATTCACAACAGAGCTGAGACCTTTGAACAGATTTACCCGTCACACAAAAACTCAGACACATTGACCTTCTGGGCCGCAATAAAGAAAAGAATGAAAATGTCTGACCCATACTCAGAAAGGAGGAAGTTGGAGAGATGCATGCCTATCGGCGAGAATCTGTGTCGGATCTTTGTGGAAGAGTATGGACTCAAGAGAGGGGTGCAAGTGGACATAGAGAGCACTGAAAGAGAATTCTTGCTTAAAAGAGTTGAAAAGGCAAAGAAGATGATTGAAGCTCACTCTGAAAGGTCAGACCCCGACTGGATGGTCAATCATTTCTTCTTATTCATGAAAACTCAGCTGTGCACCAAATTTGAAAAAAGGTTCGCTGACGCAAAAGCCGGGCAAACTTTGGCGTGCTTCTCACATCAGGTTCTAGCTAGATTTGGGGTTCCTATCAGAGTGGCTGAAAAAAAGTTAAGGGTGCAGCTTGGTGAAAATGTCTACATACACTCTGGAAAGCAGCTGGATGAACTCAATGAATGGTGCATCAAGTTTTCCACAGGTTTTGGGACAGATTCTGATTACGAAAGTTTTGATCGGTCACAAGATGCTCTTGTTCTAGCTTTTGAAATCCATTTGCTAAGATTTCTGGGATGGAGTACTGACCAGGTTGAGGATTACACAGCACTGAAATTGAGATTGGGGTGCCGATTGGGCTACCTTGCAATAATGAGGTTCACCGGGGAGTTTGGCACCTTCTTTCTTAACACATGTTGCAACATGTTGTTCACATGTCTGAGGTATAAAATAAACAGAAGGACCCCAATAGCTTTCGCTGGTGATGATATGTTTAGTCCAGGGCGTCTTGAAGTGAGAAAGGATAGAGAGTTTCTTCTGAATCGGTTTAGCTTGAAGGCAAAGGTGAACTTCTCGAAAGAACCAATTTTTTGCGGCTGGAGGATGACTCCGTATGGAATAGTTAAGGAGCCAAAGCTGGTGCTTGAAAGGTTTAAGATAGCCGAGGAACGAGGTTGTTTCAAGGAATGCCTCATCAACTACTGTCTCGAGGTTTCCTTTGCCTACAGATTAGGTGAGAGGCTGTATGATGTGATTAAGAACATTCAGGATCAACAAGCTCTTGTGAGAATAGTTGTGAAGAACAAAAAATTTCTTCCAAAGAAAATCAGAAAGGAATTTGAATCCTACGACCATGAGTTTGGTGGATGTGAATCAGTTCAGGAGGAAGGTGAATGCGACAGGGTCCATCGCTTCGGCTGTGGACTCGAATGCAATATATAAGCTGTCTCCCTTCCACGCTCTGCAGTCTGACACTTATGTCAGGAAATCTGAATTTAAGGTCAAGATGCAACAGACTAATGAGCGTGGCGTGGCTTGCATGTCTGTGCCACTGTTTGATGACCATGACAAACAGACAATCCGGGAATCCAAATTACCATATGTACATATAGCAGTTGTTTTGATTCAAATATCATGTTTGTTTGATTGGGCCATGTCTGAAGGAATGGAGGGAACCTTTGCCTTGATGGACACGTTGTTCGATAATGTCCAGGACAATGTCATAAGGGCCTGCAATTTCCGGTTTGTAGAAGGTCGTGCTGCTTGCTGCTTCAAACTGAACTTTCCAGTCTGTTCCGAGGATGCTTTAGGGGGAAGGCCAATCGTGCCTTACATTAAGGTGGTTGGTGCAAACATCAGGGAGGGGTTGAAGGGCTTCTCTGTTTCAGTTGGAACGATTTTCTCACTGAACAAAACGGAATTCCCTTCTGTCTCAATGAGGATCGATCACGACTTTGTCAATATAGTTGGGACTGAATTTCTTCCCAAAGAAAAGTTAACCGAGTTGACTTATGAGGAAATTTCAGATTCCTTTCAGAGGTTGCAAACCCTCCCAAAACCCGTCACGGCTTCGCTATCCAGAGGAAGGAAAGAACCCATTCGAGTAAGAGATTACTCAGTTATGTCGAGTGCTAGATGTTTGATTGATAAAGATGGGTCTGACAAAGAAGGAAAGGTCAGAGATGAGGAAGGAAGTCGACGCTCTACTGATAACGAAATTCAAACCGCAAGTGAAGCAGGGCCAGCAGTTCTCCGACGGAGCTCAAGCTTTTCTCAAGGAGATGGTGTTTGGGAACATAGCTCTCAAAGGAGCAAGCGAACAGACAGAGTTCGAAGACCAAGAAGTGTCAAGCGGTTGGTTCGTTCCGGGGTTAGGGGGGCGCCACGGACAGATGGATCTGGAGTTAGAGTTTCCAGGGGAGGAGAACAAAGCGCTCCGGGACGGGGCATACAAGTTCAGGGTGAACTTCTTCGGTCTGGTTCAGAGTCTGACAGCGCTGCTTAGGGGCAGCACTAGTGTGTTTGTAAATAATAAGCCTTTGAGGCGTATGTGTGTGGCCTATGCGTCCGAGGCCAAAGCCTATTTAGAGGCTAAGAAAGTGGACGGTGAGTTCTCAAATCTGGTTCTCAAGATGCCTGCGACATGCAAGCATGCTCCAGAAGTTTGCTTTGATTTCAATGAAGGTTTGGATGTCTTGCGTCTTACTGACGTGCAAGCTCAGACAATGCAGAGGTTGTCTCGTAGACTTTTTGCTACGGAACTGAAGAAGCGTGAGTCTGAGAGGTCCGTGGAGGATCACATCGGTGACCAGGTGTAGTTTGGTCCAGGTTTCTTCAAATCTGGCGGCTTAGCTTTAACTTAATAAAGTGTGCCTTCTGTTTAAGAATTTTCAG' # @param {type:"string"}
!printf ">%s\n%s\n" "seq" $fasta > sequences.fasta
!makeblastdb -in sequences.fasta -dbtype nucl -out my_blast_db
!blastn -num_threads 8 -query Spades_Result/scaffolds.fasta -db my_blast_db -out blast_results.txt -outfmt 6
from Bio import SeqIO

# File paths
blast_results_file = 'blast_results.txt'  # Contains sequence identifiers
fasta_file = 'Spades_Result/scaffolds.fasta'  # Contains the sequences
output_file = 'extracted_sequences.fasta'  # Output file for extracted sequences

# Step 1: Extract sequence identifiers from BLAST results
def extract_identifiers(blast_file):
    identifiers = set()
    with open(blast_file, 'r') as f:
        for line in f:
            if not line.startswith('#'):  # Skip comment lines
                parts = line.split()
                # Assuming the identifier is in the first column
                identifier = parts[0]
                identifiers.add(identifier)
    return identifiers

# Step 2: Extract matching sequences from the FASTA file
def extract_sequences(fasta_file, identifiers, output_file):
    with open(output_file, 'w') as out_file:
        for record in SeqIO.parse(fasta_file, 'fasta'):
            if record.id in identifiers:
                SeqIO.write(record, out_file, 'fasta')

# Run the extraction
identifiers = extract_identifiers(blast_results_file)
extract_sequences(fasta_file, identifiers, output_file)

print(f'Sequences extracted to {output_file}')